In [ ]:
import numpy as np
import biotite.structure as struc
import biotite.structure.io.pdb as pdb
import biotite.structure.io.pdbx as pdbx
from biotite.structure.io.pdbx import CIFFile
from biotite.structure.io.pdb import PDBFile
from biotite.structure.io import save_structure
import os
import gzip
import warnings

from biotite.structure.io.pdbx.convert import (
    _parse_intra_residue_bonds, 
    _parse_inter_residue_bonds, 
    _filter, 
    _get_block
)

PDB_ROOT = '/mnt/data/wwpdb/data/structures/all'

def parse_complete_cif(cif_path, dump_pdb_path=None, save_mode='all', sanity_check=True):

    sanity_failed = []
    if cif_path.endswith('.cif.gz'):
        cif_file = CIFFile.read(gzip.open(cif_path, "rt"))
    elif cif_path.endswith('.cif'):
        cif_file = CIFFile.read(cif_path)
    else:
        raise ValueError("Unsupported file format")

    # 首先 需要 altloc=all 这会读取每个 model 当中的所有原子 包括位置不确定的altloc
    # biotite 目前的读取逻辑不支持在读取所有 altloc 的同时设置 include bonds 为 True
    # 因此 需要手动补全bond信息, 这就是整个函数的目的——在读取所有 altloc 的结构后, 根据altloc ID拆分出不同的构象, 分别计算bond信息, 最后合并回去
    # 当然这是用 pdbx 读取 cif 文件的时候,你如果用 pdb 读取 pdb 文件,虽然可以设置 include bonds 为 True 的同时读取所有 altloc,但是他的键连关系会出问题
    # 这源自 biotite 把 altloc 的原子的 atom id 解析为不同的 atom id, 导致混乱
    structure_all_altlocs = pdbx.get_structure(
        cif_file, model=1, altloc='all', 
        extra_fields=['b_factor','occupancy','atom_id','charge'],
        include_bonds=False
    )

    # 获取原始的 cif 数据块
    # 以下的逻辑其实就是 pdbx.get_structure 里面的 bond 解析逻辑
    # 我们这里主要是为了把构象分开处理 然后分别把 bond 信息 assign 回去
    block = _get_block(cif_file, None)
    atom_site_original = block.get("atom_site")
    struct_conn_original = block.get("struct_conn")
    chem_comp_bond_original = block.get("chem_comp_bond")

    if atom_site_original is None:
        raise ValueError("lack 'atom_site' in file, cannot proceed.")

    custom_bond_dict = None
    if chem_comp_bond_original:
        try:
            custom_bond_dict = _parse_intra_residue_bonds(chem_comp_bond_original)
        except KeyError:
            warnings.warn(
                "'_chem_comp_bond' lacks, use ccd.",
                UserWarning
            )

    altloc_ids = np.unique(structure_all_altlocs.altloc_id)
    conformation_ids = [alt_id for alt_id in altloc_ids if (alt_id != '.' and alt_id != ' ')]
    common_indices = np.where((structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' '))[0]

    all_bonds_list = []

    if not conformation_ids:
        bonds = struc.connect_via_residue_names(structure_all_altlocs, custom_bond_dict=custom_bond_dict)
        if struct_conn_original:
            inter_bonds = _parse_inter_residue_bonds(
                atom_site_original, struct_conn_original, atom_count=len(structure_all_altlocs)
            )
            bonds = bonds.merge(inter_bonds)
        structure_all_altlocs.bonds = bonds

    else:
        for conf_id in conformation_ids:
            conf_indices = np.where(structure_all_altlocs.altloc_id == conf_id)[0]
            temp_indices = np.concatenate([common_indices, conf_indices])
            temp_indices.sort() 
            
            temp_structure = structure_all_altlocs[temp_indices]

            temp_atom_site = _filter(atom_site_original, temp_indices)
            
            temp_bonds = struc.connect_via_residue_names(temp_structure, custom_bond_dict=custom_bond_dict)
            if struct_conn_original:
                inter_bonds = _parse_inter_residue_bonds(
                    temp_atom_site, struct_conn_original, atom_count=len(temp_structure)
                )
                temp_bonds = temp_bonds.merge(inter_bonds)

            original_indices_bonds = temp_bonds.as_array()
            if original_indices_bonds.shape[0] > 0:
                original_indices_bonds[:, 0] = temp_indices[original_indices_bonds[:, 0]]
                original_indices_bonds[:, 1] = temp_indices[original_indices_bonds[:, 1]]
                all_bonds_list.append(original_indices_bonds)

        if all_bonds_list:
            final_bond_array = np.vstack(all_bonds_list)
            final_bond_array[:, :2] = np.sort(final_bond_array[:, :2], axis=1)
            _, unique_indices = np.unique(final_bond_array, axis=0, return_index=True)
            unique_bonds = final_bond_array[np.sort(unique_indices)]
            
            final_bond_list = struc.BondList(structure_all_altlocs.array_length(), bonds=unique_bonds)
            structure_all_altlocs.bonds = final_bond_list

    if sanity_check:
        
        pdb_file = cif_path.replace('.cif.gz', '.ent.gz')
        pdb_file = pdb_file.split('mmCIF')[0] + 'pdb/pdb' + pdb_file.split('mmCIF/')[-1]
        pdb_file = PDBFile.read(gzip.open(pdb_file, "rt"))

        cif_struc_first = pdbx.get_structure(
            cif_file, model=1, altloc='first', 
            extra_fields=['b_factor','occupancy','atom_id','charge'],
            include_bonds=True
        )

        pdb_struc_first = pdb.get_structure(
            pdb_file, model=1, altloc='first', 
            extra_fields=['b_factor','occupancy','atom_id','charge'],
            include_bonds=True
        )

        if len(conformation_ids) > 0:

            structure_all_altlocs_model_A = structure_all_altlocs[(structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' ') | (structure_all_altlocs.altloc_id == 'A')]
            structure_all_altlocs_model_B = structure_all_altlocs[(structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' ') | (structure_all_altlocs.altloc_id == 'B')]

            # sanitizing model_A is the same with first altloc selection
            try:
                assert (structure_all_altlocs_model_A.bonds._bonds - cif_struc_first.bonds._bonds).max() < 1e-6
                assert (structure_all_altlocs_model_A.coord - cif_struc_first.coord).max() < 1e-6
                assert structure_all_altlocs_model_A.bonds._bonds.shape == pdb_struc_first.bonds._bonds.shape
                assert structure_all_altlocs_model_B.bonds._bonds.shape == pdb_struc_first.bonds._bonds.shape
            except AssertionError:
                sanity_failed.append(cif_path.split('/')[-1][0:4])
                print(f"Sanity check failed for {cif_path.split('/')[-1][0:4]}.")
            
        else:
            # sanitizing all altlocs is the same with first altloc selection when no altlocs
            try:
                assert (structure_all_altlocs.bonds._bonds - cif_struc_first.bonds._bonds).max() < 1e-6
                assert (structure_all_altlocs.coord - cif_struc_first.coord).max() < 1e-6
                assert structure_all_altlocs.bonds._bonds.shape == pdb_struc_first.bonds._bonds.shape
            except AssertionError:
                sanity_failed.append(cif_path.split('/')[-1][0:4])
                print(f"Sanity check failed for {cif_path.split('/')[-1][0:4]}.")

    if dump_pdb_path is not None:
        if save_mode == 'first':
            save_structure(dump_pdb_path, cif_struc_first)
        elif save_mode == 'all':
            save_structure(dump_pdb_path, structure_all_altlocs)
        elif save_mode == 'separate':
            if len(conformation_ids) == 0:
                save_structure(dump_pdb_path, cif_struc_first)
            else:
                for alt_id in conformation_ids:
                    conf_structure = structure_all_altlocs[(structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' ') | (structure_all_altlocs.altloc_id == alt_id)]
                    conf_dump_path = dump_pdb_path.replace('.pdb', f'_altloc_{alt_id}.pdb')
                    save_structure(conf_dump_path, conf_structure)
        else:
            raise ValueError("Unsupported save_mode. Choose from 'first', 'all', or 'separate'.")

    return structure_all_altlocs, sanity_failed

parse_complete_cif(
    '/mnt/data/wwpdb/data/structures/all/mmCIF/1qkt.cif.gz', 
    dump_pdb_path='/home/megagatlingpea/workdir/ProteinDesign/RAPiGen/test/1qkt_dump.pdb',
    save_mode='separate',
    sanity_check=True
)

(array([
 	Atom(np.array([ 20.149,  35.39 , 139.337], dtype=float32), chain_id="A", res_id=304, ins_code="", res_name="ASN", hetero=False, atom_name="N", element="N", atom_id=1, b_factor=68.16, occupancy=1.0, charge=0, altloc_id="."),
 	Atom(np.array([ 21.477,  36.068, 139.445], dtype=float32), chain_id="A", res_id=304, ins_code="", res_name="ASN", hetero=False, atom_name="CA", element="C", atom_id=2, b_factor=68.33, occupancy=1.0, charge=0, altloc_id="."),
 	Atom(np.array([ 21.319,  37.575, 139.646], dtype=float32), chain_id="A", res_id=304, ins_code="", res_name="ASN", hetero=False, atom_name="C", element="C", atom_id=3, b_factor=67.48, occupancy=1.0, charge=0, altloc_id="."),
 	Atom(np.array([ 21.151,  38.319, 138.671], dtype=float32), chain_id="A", res_id=304, ins_code="", res_name="ASN", hetero=False, atom_name="O", element="O", atom_id=4, b_factor=67.59, occupancy=1.0, charge=0, altloc_id="."),
 	Atom(np.array([ 22.314,  35.823, 138.178], dtype=float32), chain_id="A", res_id=304,

In [6]:
import os

cif_files = set()
for f in os.listdir('/mnt/data/wwpdb/data/structures/all/mmCIF'):
    if f.endswith('.cif.gz'):
        cif_files.add(f.replace('.cif.gz', ''))

pdb_files = set()
for f in os.listdir('/mnt/data/wwpdb/data/structures/all/pdb'):
    if f.startswith('pdb') and f.endswith('.ent.gz'):
        pdb_files.add(f.replace('pdb', '').replace('.ent.gz', ''))


In [7]:

missing_in_b = cif_files - pdb_files
extra_in_b = pdb_files - cif_files

print('缺少')
for f in sorted(missing_in_b):
    print(f)

缺少
1bos
1pdb
1vvj
1vy4
1vy5
1vy6
1vy7
2btj
2n6z
2pdb
2vvj
3bcr
3j3q
3j3y
3j6b
3j6x
3j6y
3j77
3j78
3j79
3j7o
3j7p
3j7q
3j7r
3j8h
3j92
3j9g
3j9k
3j9l
3j9m
3j9r
3j9w
3j9y
3j9z
3ja1
3jag
3jah
3jai
3jaj
3jan
3jbn
3jbo
3jbp
3jbu
3jbv
3jc1
3jc8
3jc9
3jcd
3jce
3jcj
3jcn
3jco
3jcp
3jcs
3jct
3k1q
3pdb
3rlh
3rme
3whe
4abz
4bp7
4bts
4ctf
4ctg
4d5y
4d67
4dcb
4fhn
4fqr
4frt
4l47
4l71
4lel
4lfz
4lnt
4lsk
4lt8
4mg5
4nwr
4o9y
4p6f
4p70
4pdb
4qyk
4tua
4tub
4tuc
4tud
4tue
4tvx
4u1u
4u1v
4u20
4u24
4u25
4u26
4u27
4u3m
4u3n
4u3u
4u4n
4u4o
4u4q
4u4r
4u4u
4u4y
4u4z
4u50
4u51
4u52
4u53
4u55
4u56
4u6f
4udf
4ue3
4ug0
4ujc
4ujd
4uje
4v3p
4v40
4v41
4v42
4v43
4v44
4v45
4v46
4v47
4v48
4v49
4v4a
4v4b
4v4c
4v4d
4v4e
4v4f
4v4g
4v4h
4v4i
4v4j
4v4k
4v4l
4v4m
4v4n
4v4o
4v4p
4v4q
4v4r
4v4s
4v4t
4v4u
4v4v
4v4w
4v4x
4v4y
4v4z
4v50
4v51
4v52
4v53
4v54
4v55
4v56
4v57
4v58
4v59
4v5a
4v5b
4v5c
4v5d
4v5e
4v5f
4v5g
4v5h
4v5i
4v5j
4v5k
4v5l
4v5m
4v5n
4v5o
4v5p
4v5q
4v5r
4v5s
4v5t
4v5v
4v5w
4v5x
4v5y
4v5z
4v60
4v61
4v62
4v63
4v64
4v

In [92]:
# import numpy as np
# import biotite.structure as struc
# import biotite.structure.io.pdb as pdb
# import biotite.structure.io.pdbx as pdbx
# from biotite.structure.io.pdbx import CIFFile
# from biotite.structure.io.pdb import PDBFile
# import os
# import gzip
# import warnings

# from biotite.structure.io.pdbx.convert import (
#     _parse_intra_residue_bonds, 
#     _parse_inter_residue_bonds, 
#     _filter, 
#     _get_block
# )

# PDB_ROOT = '/mnt/data/wwpdb/data/structures/all'

# try:
#     cif_file = CIFFile.read(gzip.open(os.path.join(PDB_ROOT, "mmCIF", "9yn9.cif.gz"), "rt"))
#     pdb_file = PDBFile.read(gzip.open(os.path.join(PDB_ROOT, "pdb", "pdb9yn9.ent.gz"), "rt"))
# except FileNotFoundError:
#     exit()


# structure_all_altlocs = pdbx.get_structure(
#     cif_file, model=1, altloc='all', 
#     extra_fields=['b_factor','occupancy','atom_id','charge'],
#     include_bonds=False
# )

# block = _get_block(cif_file, None)
# atom_site_original = block.get("atom_site")
# struct_conn_original = block.get("struct_conn")
# chem_comp_bond_original = block.get("chem_comp_bond")

# if atom_site_original is None:
#     raise ValueError("lack 'atom_site' in file, cannot proceed.")

# custom_bond_dict = None
# if chem_comp_bond_original:
#     try:
#         custom_bond_dict = _parse_intra_residue_bonds(chem_comp_bond_original)
#     except KeyError:
#         warnings.warn(
#             "'_chem_comp_bond' lacks, use ccd.",
#             UserWarning
#         )

# altloc_ids = np.unique(structure_all_altlocs.altloc_id)
# conformation_ids = [alt_id for alt_id in altloc_ids if (alt_id != '.' and alt_id != ' ')]
# common_indices = np.where((structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' '))[0]

# all_bonds_list = []

# if not conformation_ids:
#     bonds = struc.connect_via_residue_names(structure_all_altlocs, custom_bond_dict=custom_bond_dict)
#     if struct_conn_original:
#         inter_bonds = _parse_inter_residue_bonds(
#             atom_site_original, struct_conn_original, atom_count=len(structure_all_altlocs)
#         )
#         bonds = bonds.merge(inter_bonds)
#     structure_all_altlocs.bonds = bonds

# else:
#     for conf_id in conformation_ids:
#         conf_indices = np.where(structure_all_altlocs.altloc_id == conf_id)[0]
#         temp_indices = np.concatenate([common_indices, conf_indices])
#         temp_indices.sort() 
        
#         temp_structure = structure_all_altlocs[temp_indices]

#         temp_atom_site = _filter(atom_site_original, temp_indices)
        
#         temp_bonds = struc.connect_via_residue_names(temp_structure, custom_bond_dict=custom_bond_dict)
#         if struct_conn_original:
#             inter_bonds = _parse_inter_residue_bonds(
#                 temp_atom_site, struct_conn_original, atom_count=len(temp_structure)
#             )
#             temp_bonds = temp_bonds.merge(inter_bonds)

#         original_indices_bonds = temp_bonds.as_array()
#         if original_indices_bonds.shape[0] > 0:
#             original_indices_bonds[:, 0] = temp_indices[original_indices_bonds[:, 0]]
#             original_indices_bonds[:, 1] = temp_indices[original_indices_bonds[:, 1]]
#             all_bonds_list.append(original_indices_bonds)

#     if all_bonds_list:
#         final_bond_array = np.vstack(all_bonds_list)
#         final_bond_array[:, :2] = np.sort(final_bond_array[:, :2], axis=1)
#         _, unique_indices = np.unique(final_bond_array, axis=0, return_index=True)
#         unique_bonds = final_bond_array[np.sort(unique_indices)]
        
#         final_bond_list = struc.BondList(structure_all_altlocs.array_length(), bonds=unique_bonds)
#         structure_all_altlocs.bonds = final_bond_list
    
    

In [93]:
# cif_struc_first = pdbx.get_structure(
#     cif_file, model=1, altloc='first', 
#     extra_fields=['b_factor','occupancy','atom_id','charge'],
#     include_bonds=True
# )

# pdb_struc_first = pdb.get_structure(
#     pdb_file, model=1, altloc='first', 
#     extra_fields=['b_factor','occupancy','atom_id','charge'],
#     include_bonds=True
# )

In [94]:
# structure_all_altlocs_model_A = structure_all_altlocs[(structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' ') | (structure_all_altlocs.altloc_id == 'A')]
# structure_all_altlocs_model_B = structure_all_altlocs[(structure_all_altlocs.altloc_id == '.') | (structure_all_altlocs.altloc_id == ' ') | (structure_all_altlocs.altloc_id == 'B')]

In [95]:
# # sanitizing model_A is the same with first altloc selection
# assert (structure_all_altlocs_model_A.bonds._bonds - cif_struc_first.bonds._bonds).max() < 1e-6
# assert (structure_all_altlocs_model_A.coord - cif_struc_first.coord).max() < 1e-6
# assert structure_all_altlocs_model_A.bonds._bonds.shape == pdb_struc_first.bonds._bonds.shape
